# Jupyter Books

## 1. Installation

To install the Jupyter Book command-line interface (CLI), use `pip`!

In [ ]:
import sys

#install jupyter-book
cmd = f'{sys.executable} -m pip show jupyter-book'
cmdOutput = !{cmd}
if len(cmdOutput) > 0 and '0.6.4' in cmdOutput[1]:
    print('Jupyter-book required version is already installed!')
else:
    !pip install jupyter-book

## 2. Create a new book

Create a book using your own notebooks and markdown pages:

<span style="color:red">Note:</span> Notebook and markdown filenames cannot contain spaces

In [ ]:
import os

overwrite = False
book_name = input('Please provide a book name: ') 

if (os.path.exists(book_name)):
    new_book_name = input('A folder named ' + book_name + ' already exists. Enter a new name or the same name to overwrite the existing folder.\n')
    if book_name == new_book_name:
        overwrite = True
    book_name = new_book_name

content_folder = input('Please provide the path to your folder containing notebooks and markdown files: ')

while (not os.path.exists(content_folder)):
    content_folder = input('Cannot find folder ' + content_folder + '. Please provide another path: ')
 
if overwrite:
    !jupyter-book create "$book_name" --content-folder "$content_folder" --overwrite
else:
    !jupyter-book create "$book_name" --content-folder "$content_folder"


### <span style="color:red">Issue:</span> generate_toc.py is missing

Jupyter Book uses the Table of Contents to define the structure of your book. For example, your chapters, sub-chapters, etc.

Need to manually modify the Table of Contents (located here: mybookname/_data/toc.yml) following structure below:

```
- title: mytitle   # Title of chapter or section
  url: /myurl  # URL of section relative to the /content/ folder.
  sections:  # Contains a list of more entries that make up the chapter's sections
  not_numbered: true  # if the section shouldn't have a number in the sidebar
      (e.g. Introduction or appendices)
  expand_sections: true  # if you'd like the sections of this chapter to always
      be expanded in the sidebar.
  external: true  # Whether the URL is an external link or points to content in the book
```

Example from demo book:

```
- title: Getting started
  url: /guide/01_overview
  not_numbered: true
  expand_sections: true
  sections:
  - title: Create your book
    url: /guide/02_create
  - title: Build and publish your book
    url: /guide/03_build
  - title: FAQ
    url: /guide/04_faq
  - title: How-to and advanced topics
    url: /guide/05_advanced
```

## 3. Build
**Note**: Please update the toc.yml file with correct info (refer above to update relative paths and titles) before running the build command. Once you update the toc.yml, update _config file with appropriate info as well. <span style="color:red">If you fail to do this, **build will fail**.</span>

Build command to run:

In [ ]:
!jupyter-book build "$book_name"


**Run the below cell and click on the link to view your book in Azure Data Studio.**

In [ ]:
import re
from IPython.display import *
display(HTML("<h2><b><a href=\"command:notebook.command.openCreatedBook?&quot;"+str(re.escape(book_name))+"&quot;\">Click here to open your Book in ADS</a></b></h2>"))

## That's it!
You can now either push your book to GitHub and view with Github Pages or open it in Azure Data Studio.
